# XGBoost Hyperparameters


In [1]:
import pandas as pd

df = pd.read_csv("heart_disease.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [4]:
# Build a classifier to obtain a baseline score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Initialize the classifier with booster='gbtree' and objective='binary:logistic'
model = XGBClassifier(booster="gbtree", objective="binary:logistic", random_state=2)

# Score the baseline model using cross_val_scores and numpy
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(model, X, y, cv=5)
print("Accuracy:", np.round(scores, 2))
print("Mean Accuracy:", np.round(np.mean(scores), 2))

Accuracy: [0.82 0.8  0.77 0.78 0.77]
Mean Accuracy: 0.79


In [5]:
# Using StratifiedKFold helps ensure that the folds are representative of the whole dataset
# This ensures consistent results across cross_val_score, GridSearchCV, and RandomizedSearchCV.
# Import and initialize the StratifiedKFold object
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [6]:
# Score the baseline model with consistent folds.
scores = cross_val_score(model, X, y, cv=kfold)
print("Accuracy:", np.round(scores, 2))
print("Mean Accuracy:", np.round(np.mean(scores), 2))

Accuracy: [0.72 0.77 0.75 0.85 0.87]
Mean Accuracy: 0.79


In [8]:
# Combining GridSearchCV with RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


# Define a grid_search function with the params dictionary as input, along with random=False
def grid_search(params, random=False):
    xgb = XGBClassifier(booster="gbtree", objective="binary:logistic", random_state=2)
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=20, n_jobs=-1)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    grid.fit(X, y)
    best_params = grid.best_params_
    print("Best params:", best_params)
    best_score = grid.best_score_
    print("Best score:", np.round(best_score, 3))

In [9]:
# Tune n_estimators
# One strategy for finding an ideal value for n_estimators is using early stopping.
grid_search(params={"n_estimators": [100, 200, 400, 800]})

Best params: {'n_estimators': 400}
Best score: 0.796


In [10]:
# Tune learning_rate
grid_search(params={"learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

Best params: {'learning_rate': 0.05}
Best score: 0.802


In [11]:
# Tune max_depth, the length of the tree. The default value is 6.
grid_search(params={"max_depth": [2, 3, 5, 6, 8]})

Best params: {'max_depth': 6}
Best score: 0.793


In [12]:
# Tune gamma, the Lagrange multiplier.
# gamma provides a threshold that nodes must surpass before making further splits on a branch of the tree.
grid_search(params={"gamma": [0, 0.1, 0.5, 1, 2, 5]})

Best params: {'gamma': 0}
Best score: 0.793


In [13]:
# Tune min_child_weight, the minimum number of instances required for a node to be split.
# min_child_weight reduces overfitting by increasing its value.
grid_search(params={"min_child_weight": [1, 2, 3, 4, 5]})

Best params: {'min_child_weight': 3}
Best score: 0.799


In [14]:
# Tune subsample, the fraction of samples used for fitting the individual base learners.
grid_search(params={"subsample": [0.5, 0.7, 0.8, 0.9, 1]})

Best params: {'subsample': 0.5}
Best score: 0.796


In [15]:
# Tune colsample_bytree, the fraction of features used for fitting the individual base learners.
# colsample_bytree limits the influence of columns and reduces variance.
grid_search(params={"colsample_bytree": [0.5, 0.7, 0.8, 0.9, 1]})

Best params: {'colsample_bytree': 0.9}
Best score: 0.796


## Applying early stopping

`early_stopping_rounds` is not a hyperparameter, but a strategy for optimizing the `n_estimators` hyperparameter. Early stopping allows us to set a large value for `n_estimators` without worrying about overfitting. The algorithm will stop iterating when the validation score stops improving, even if we aren't at the hard stop for `n_estimators`. This is a very useful feature for avoiding overfitting!


In [17]:
# Display an evaluation metric for each round of training with the default number of estimators.
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

# Initialize the model
model = XGBClassifier(booster="gbtree", objective="binary:logistic", random_state=2)

# Declare eval_set
eval_set = [(X_test, y_test)]

# Declare eval_metric
eval_metric = "error"

# Fit the model with eval_metric and eval_set
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set)

# Check the final score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", np.round(accuracy * 100.0, 3))

[0]	validation_0-error:0.27632
[1]	validation_0-error:0.15789
[2]	validation_0-error:0.07895
[3]	validation_0-error:0.09211
[4]	validation_0-error:0.10526
[5]	validation_0-error:0.10526
[6]	validation_0-error:0.10526
[7]	validation_0-error:0.13158
[8]	validation_0-error:0.13158
[9]	validation_0-error:0.13158
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.14474
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.15789
[14]	validation_0-error:0.15789
[15]	validation_0-error:0.15789
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.15789
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.17105
[23]	validation_0-error:0.15789
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.17105
[28]	validation_0-error:0.17105
[29]	validation_0-error:0.17105
[30]	validation_0-error:0.17105
[31]	validation_0-

/Users/graftoncook/.pyenv/versions/3.10.9/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


In [19]:
# early_stopping_rounds is an optional parameter to include with eval_set and eval_metric.
# Repeat the previous code with early_stopping_rounds=10 added in
model = XGBClassifier(booster="gbtree", objective="binary:logistic", random_state=2)
eval_set = [(X_test, y_test)]
eval_metric = "error"
model.fit(
    X_train,
    y_train,
    eval_metric=eval_metric,
    eval_set=eval_set,
    early_stopping_rounds=10,
    verbose=True,
)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", np.round(accuracy * 100.0, 3))

[0]	validation_0-error:0.27632
[1]	validation_0-error:0.15789
[2]	validation_0-error:0.07895
[3]	validation_0-error:0.09211
[4]	validation_0-error:0.10526
[5]	validation_0-error:0.10526
[6]	validation_0-error:0.10526
[7]	validation_0-error:0.13158
[8]	validation_0-error:0.13158
[9]	validation_0-error:0.13158
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.14474
Accuracy: 92.105


/Users/graftoncook/.pyenv/versions/3.10.9/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/graftoncook/.pyenv/versions/3.10.9/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [21]:
model = XGBClassifier(random_state=2, n_estimators=5000)
eval_set = [(X_test, y_test)]
eval_metric = "error"
model.fit(
    X_train,
    y_train,
    eval_metric=eval_metric,
    eval_set=eval_set,
    early_stopping_rounds=100,
)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[0]	validation_0-error:0.27632
[1]	validation_0-error:0.15789
[2]	validation_0-error:0.07895
[3]	validation_0-error:0.09211
[4]	validation_0-error:0.10526
[5]	validation_0-error:0.10526
[6]	validation_0-error:0.10526
[7]	validation_0-error:0.13158
[8]	validation_0-error:0.13158
[9]	validation_0-error:0.13158
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.14474
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.15789
[14]	validation_0-error:0.15789
[15]	validation_0-error:0.15789
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.15789
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.17105
[23]	validation_0-error:0.15789
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.17105
[28]	validation_0-error:0.17105
[29]	validation_0-error:0.17105
[30]	validation_0-error:0.17105
[31]	validation_0-

/Users/graftoncook/.pyenv/versions/3.10.9/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/graftoncook/.pyenv/versions/3.10.9/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[80]	validation_0-error:0.14474
[81]	validation_0-error:0.14474
[82]	validation_0-error:0.15789
[83]	validation_0-error:0.15789
[84]	validation_0-error:0.15789
[85]	validation_0-error:0.14474
[86]	validation_0-error:0.14474
[87]	validation_0-error:0.14474
[88]	validation_0-error:0.15789
[89]	validation_0-error:0.14474
[90]	validation_0-error:0.14474
[91]	validation_0-error:0.14474
[92]	validation_0-error:0.14474
[93]	validation_0-error:0.14474
[94]	validation_0-error:0.14474
[95]	validation_0-error:0.14474
[96]	validation_0-error:0.14474
[97]	validation_0-error:0.15789
[98]	validation_0-error:0.15789
[99]	validation_0-error:0.15789
[100]	validation_0-error:0.15789
[101]	validation_0-error:0.15789
[102]	validation_0-error:0.15789
Accuracy: 92.11%


## Combining hyperparameters

There is no single right way to fine-tune hyperparameters. A good systematic approach is to add one hyperparameter at a time and aggregate results along the way.


In [22]:
# Start with n_estimators
grid_search(params={"n_estimators": [2, 25, 50, 75, 100]})

Best params: {'n_estimators': 100}
Best score: 0.793


In [24]:
# Move on to max_depth
grid_search(params={"max_depth": [1, 2, 3, 4, 5, 6, 7, 8], "n_estimators": [100]})

Best params: {'max_depth': 1, 'n_estimators': 100}
Best score: 0.812


In [25]:
# A limitation with keeping only the top values is that we may miss out on better combinations.
grid_search(
    params={"max_depth": [2, 3, 4, 5, 6, 7, 8], "n_estimators": [50, 75, 100, 125, 150]}
)

Best params: {'max_depth': 2, 'n_estimators': 50}
Best score: 0.806


In [26]:
# Tune learning_rate
grid_search(
    params={
        "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
        "max_depth": [2],
        "n_estimators": [50],
    }
)

Best params: {'learning_rate': 0.5, 'max_depth': 2, 'n_estimators': 50}
Best score: 0.809


In [27]:
# Tune min_child_weight
grid_search(
    params={
        "min_child_weight": [1, 2, 3, 4, 5],
        "learning_rate": [0.5],
        "max_depth": [2],
        "n_estimators": [50],
    }
)

Best params: {'learning_rate': 0.5, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 50}
Best score: 0.809


In [28]:
# min_child_weight=1 is the default, so no point in tuning that further.
# Tune subsample
grid_search(
    params={
        "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1],
        "learning_rate": [0.5],
        "max_depth": [2],
        "n_estimators": [50],
    }
)

Best params: {'learning_rate': 0.5, 'max_depth': 2, 'n_estimators': 50, 'subsample': 1}
Best score: 0.809


In [30]:
# No improvement with the last few hyperparameters.
# Recall that two estimators gave the best results, so let's try a RandomizedSearchCV including that value.
grid_search(
    params={
        "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1],
        "min_child_weight": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        "learning_rate": [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
        "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, None],
        "n_estimators": [2, 25, 50, 75, 100, 150, 175, 200, 250],
    },
    random=True,
)

Best params: {'subsample': 0.9, 'n_estimators': 150, 'min_child_weight': 6, 'max_depth': 1, 'learning_rate': 0.1}
Best score: 0.822


In [33]:
# Let's use the best parameters from the RandomizedSearchCV moving forward and add the colsample hyperparameters
grid_search(
    params={
        "subsample": [0.9],
        "min_child_weight": [6],
        "learning_rate": [0.1],
        "max_depth": [1],
        "colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9, 1],
        "n_estimators": [150],
    }
)

Best params: {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 1, 'min_child_weight': 6, 'n_estimators': 150, 'subsample': 0.9}
Best score: 0.829


In [34]:
# colsample_bylevel
grid_search(
    params={
        "subsample": [0.9],
        "min_child_weight": [6],
        "learning_rate": [0.1],
        "max_depth": [1],
        "colsample_bytree": [0.6],
        "colsample_bylevel": [0.5, 0.6, 0.7, 0.8, 0.9, 1],
        "n_estimators": [150],
    }
)

Best params: {'colsample_bylevel': 0.5, 'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 1, 'min_child_weight': 6, 'n_estimators': 150, 'subsample': 0.9}
Best score: 0.842


In [37]:
# colsample_bynode
grid_search(
    params={
        "subsample": [0.9],
        "min_child_weight": [6],
        "learning_rate": [0.1],
        "max_depth": [1],
        "colsample_bytree": [0.6],
        "colsample_bylevel": [0.5],
        "colsample_bynode": [0.5, 0.6, 0.7, 0.8, 0.9, 1],
        "n_estimators": [150],
    }
)

Best params: {'colsample_bylevel': 0.5, 'colsample_bynode': 1, 'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 1, 'min_child_weight': 6, 'n_estimators': 150, 'subsample': 0.9}
Best score: 0.842


In [38]:
# Let's try all three colsample hyperparameters together to see whether we can improve the score.
grid_search(
    params={
        "subsample": [0.9],
        "min_child_weight": [6],
        "learning_rate": [0.1],
        "max_depth": [1],
        "colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9, 1],
        "colsample_bylevel": [0.5, 0.6, 0.7, 0.8, 0.9, 1],
        "colsample_bynode": [0.5, 0.6, 0.7, 0.8, 0.9, 1],
        "n_estimators": [150],
    }
)

Best params: {'colsample_bylevel': 0.5, 'colsample_bynode': 1, 'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 1, 'min_child_weight': 6, 'n_estimators': 150, 'subsample': 0.9}
Best score: 0.842


In [39]:
# 5% improvement in accuracy with the best parameters. Let's tune gamma.
grid_search(
    params={
        "subsample": [0.9],
        "min_child_weight": [6],
        "learning_rate": [0.1],
        "max_depth": [1],
        "colsample_bytree": [0.6],
        "colsample_bylevel": [0.5],
        "colsample_bynode": [1],
        "n_estimators": [150],
        "gamma": [0, 0.01, 0.05, 0.1, 0.5, 1, 2, 3],
    }
)

Best params: {'colsample_bylevel': 0.5, 'colsample_bynode': 1, 'colsample_bytree': 0.6, 'gamma': 1, 'learning_rate': 0.1, 'max_depth': 1, 'min_child_weight': 6, 'n_estimators': 150, 'subsample': 0.9}
Best score: 0.842
